In [94]:
import os
import json
import cv2
import numpy as np
import imageio.v3 as iio
import trimesh

In [95]:
def load_camera_params(json_path):
    with open(json_path, 'r') as f:
        cam = json.load(f)
    K = np.array(cam['intrinsics'])
    c2w = np.array(cam['c2w'])
    return K, c2w

def backproject_to_world(depth, K, c2w):
    H, W = depth.shape
    i, j = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
    pixels = np.stack((i, j, np.ones_like(i)), axis=-1).reshape(-1, 3).T  # (3, H*W)

    # Compute direction vectors in camera coordinates (not normalized)
    K_inv = np.linalg.inv(K)
    rays = K_inv @ pixels  # (3, N)

    # Do not normalize rays — this is key
    # Instead, just scale Z component of the rays to match depth:
    scale = depth.reshape(-1) / rays[2, :]  # Z_depth / ray_z_component
    cam_points = rays * scale  # scale each ray appropriately

    # Convert to homogeneous coordinates
    cam_points = np.vstack([cam_points, np.ones((1, cam_points.shape[1]))])  # (4, N)

    # Transform to world space
    world_coords = (c2w @ cam_points).T[:, :3]  # (N, 3)
    return world_coords


def convert_pose(C2W):
    flip_yz = np.eye(4)
    flip_yz[1, 1] = -1
    flip_yz[2, 2] = -1
    C2W = np.matmul(C2W, flip_yz)
    return C2W

# Load data
# data_dir = "/Users/gemmechu/Documents/meta/data/cube/camera_test"
# K, c2w = load_camera_params(os.path.join(data_dir, "camera.json"))

# c2w  = convert_pose(c2w)
# depth = iio.imread(os.path.join(data_dir, "depth_0002.exr"))[:,:,0]  # shape (H, W), float32
# rgb = iio.imread(os.path.join(data_dir, "rgb.png"))[:, :, :3]  # (H, W, 3)

# # Convert depth from 1-channel to linear depth if necessary (usually linear in EXR)
# points_3d = backproject_to_world(depth, K, c2w)

In [53]:
rgb = iio.imread(os.path.join(data_dir, "rgb.png"))
rgb.shape

(500, 500)

In [59]:

cloud = trimesh.PointCloud(vertices=points_3d)
a = cloud.export(os.path.join(data_dir, "points2.ply"))


In [42]:
def project_3d_point_to_image(point_3d, K, C2W):
    a = np.eye(4)
    a[:3, :3] = K
    K = a
    # Create a 4x4 transformation matrix
    extrinsic_matrix = np.linalg.inv(C2W)
    # Project the 3D point onto the image
    homogeneous_point_3d = np.append(point_3d, 1)
    projected_point_homogeneous = K @ extrinsic_matrix @ homogeneous_point_3d

    # Normalize the homogeneous coordinates
    projected_point = projected_point_homogeneous[:2] / projected_point_homogeneous[2]

    # Extract the x and y coordinates of the projected point
    x_projected, y_projected = projected_point
    return int(y_projected),int(x_projected)
K, c2w = load_camera_params(os.path.join(data_dir, "camera.json"))

point_3d, K, C2W = [0, 1, 1], K, c2w
project_3d_point_to_image(point_3d, K, C2W)

(177, 400)

In [96]:
cameras_dict = "/Users/gemmechu/Documents/meta/data/cube/cameras.json"
data = json.load(open(cameras_dict))
frames = data["frames"]
frame = frames["0070"]
K = np.array(data["intrinsics"])
data_dir = "/Users/gemmechu/Documents/meta/data/cube"

In [97]:
c2w = np.array(frame["c2w"])
# c2w  = convert_pose(c2w)
depth = iio.imread(os.path.join(data_dir, "depth", frame["depth"]))[:,:,0]  # shape (H, W), float32 

points_3d = backproject_to_world(depth, K, c2w)
cloud = trimesh.PointCloud(vertices=points_3d)
a = cloud.export(os.path.join(data_dir,"points_inv", "points_20.ply"))


In [110]:
cameras_dict = "/Users/gemmechu/Documents/meta/data/cube/cameras.json"
data = json.load(open(cameras_dict))
frames = data["frames"]
K = np.array(data["intrinsics"])
data_dir = "/Users/gemmechu/Documents/meta/data/cube"
outdir = os.path.join
for fname in frames:
    frame = frames[fname]
    c2w = np.array(frame["c2w"])
    rgb = iio.imread(os.path.join(data_dir, "rgb",frame["rgb"]))[:,:,:3]
    c2w  = convert_pose(c2w)
    depth = iio.imread(os.path.join(data_dir, "depth", frame["depth"]))[:,:,0]  # shape (H, W), float32 

    points_3d = backproject_to_world(depth, K, c2w)

    colors = rgb.reshape(-1, 3)
    if colors.dtype != np.uint8:
        colors = (np.clip(colors, 0, 1) * 255).astype(np.uint8)

    # Create point cloud with colors
    cloud = trimesh.PointCloud(vertices=points_3d, colors=colors)
    # cloud = trimesh.PointCloud(vertices=points_3d)
    outpath = os.path.join(data_dir,"points_opengl",fname+".ply" )
    a = cloud.export(outpath)
    

In [109]:
python3 data_downloader/dataset_downloader_base_main.py -c /Users/gemmechu/Documents/meta/code/hot3d/hot3d-aria_download_urls.json -o /Users/gemmechu/Documents/meta/code/hot3d/dataset --sequence_name P0005_96f32b8f --data_types all

(500, 500, 4)